In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
sys.path.append("..")

In [8]:
from optimus import Optimus
op = Optimus()

In [9]:
op.spark

In [11]:
df =op.load.url("https://raw.githubusercontent.com/ironmussa/Optimus/master/examples/foo.csv")

In [12]:
df.table()

id 1 (int),firstName 2 (string),lastName 3 (string),billingId 4 (int),product 5 (string),price 6 (int),birth 7 (string),dummyCol 8 (string)
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down


In [6]:
df.show()

+---+--------------------+--------------------+---------+----------+-----+----------+--------+
| id|           firstName|            lastName|billingId|   product|price|     birth|dummyCol|
+---+--------------------+--------------------+---------+----------+-----+----------+--------+
|  1|                Luis|         Alvarez$$%!|      123|      Cake|   10|1980/07/07|   never|
|  2|               André|              Ampère|      423|      piza|    8|1950/07/08|   gonna|
|  3|               NiELS|          Böhr//((%%|      551|     pizza|    8|1990/07/09|    give|
|  4|                PAUL|              dirac$|      521|     pizza|    8|1954/07/10|     you|
|  5|              Albert|            Einstein|      634|     pizza|    8|1990/07/11|      up|
|  6|             Galileo|             GALiLEI|      672|     arepa|    5|1930/08/12|   never|
|  7|                CaRL|            Ga%%%uss|      323|      taco|    3|1970/07/13|   gonna|
|  8|               David|          H$$$ilbert|   

In [12]:
# This is a custom function
def num_to_string(value, arg):
    return "this was a number"
    
df\
    .rows.sort("product","desc")\
    .cols.lower(["firstName","lastName"])\
    .cols.date_transform("birth", "new_date", "yyyy/MM/dd", "dd-MM-YYYY")\
    .cols.years_between("birth", "years_between", "yyyy/MM/dd")\
    .cols.remove_accents("lastName")\
    .cols.remove_special_chars(["firstName","lastName"])\
    .cols.replace("product","taaaccoo","taco")\
    .cols.replace("product",["piza","pizzza"],"pizza")\
    .rows.drop(df["id"]<7)\
    .cols.drop("dummyCol")\
    .cols.rename(str.lower)\
    .cols.apply_by_dtypes("product",num_to_string,"string", data_type="integer")\
    .cols.trim("*")\
    .rows.sort("id")\
    .show()

+---+---------+--------+---------+-----------------+-----+----------+----------+-------------+
| id|firstname|lastname|billingid|          product|price|     birth|  new_date|years_between|
+---+---------+--------+---------+-----------------+-----+----------+----------+-------------+
| 10|    james| maxwell|      875|             taco|    3|1923/03/12|12-03-1923|      95.4355|
| 11|    isaac|  newton|      992|            pasta|    9|1999/02/15|15-02-1999|      19.5108|
| 12|     emmy| noether|      234|            pasta|    9|1993/12/08|08-12-1993|      24.6962|
| 13|      max|  planck|      111|       hamburguer|    4|1994/01/04|04-01-1994|      24.6237|
| 14|     fred|   hoyle|      553|            pizza|    8|1997/06/27|27-06-1997|      21.1452|
| 15| heinrich|   hertz|      116|            pizza|    8|1956/11/30|30-11-1956|      61.7204|
| 16|  william| gilbert|      886|             BEER|    2|1958/03/26|26-03-1958|      60.3978|
| 17|    marie|   curie|      912|             Ric

In [18]:
df.show()
def func(value, args):
    return value +args[0] + args[1]


df.cols.apply("billingid",func,"int", [1,2]).show()

#df.cols.apply(["num", "new_col_1"], func, "int", 32 ,"udf").table()

+---+--------------------+--------------------+---------+----------+-----+----------+--------+
| id|           firstName|            lastName|billingId|   product|price|     birth|dummyCol|
+---+--------------------+--------------------+---------+----------+-----+----------+--------+
|  1|                Luis|         Alvarez$$%!|      123|      Cake|   10|1980/07/07|   never|
|  2|               André|              Ampère|      423|      piza|    8|1950/07/08|   gonna|
|  3|               NiELS|          Böhr//((%%|      551|     pizza|    8|1990/07/09|    give|
|  4|                PAUL|              dirac$|      521|     pizza|    8|1954/07/10|     you|
|  5|              Albert|            Einstein|      634|     pizza|    8|1990/07/11|      up|
|  6|             Galileo|             GALiLEI|      672|     arepa|    5|1930/08/12|   never|
|  7|                CaRL|            Ga%%%uss|      323|      taco|    3|1970/07/13|   gonna|
|  8|               David|          H$$$ilbert|   

In [24]:
from pyspark.sql import functions as F

def func(col_name, args):
    return F.col(col_name)/20

df.cols.apply_expr("billingid", func, 20).show()

+---+--------------------+--------------------+---------+----------+-----+----------+--------+
| id|           firstName|            lastName|billingid|   product|price|     birth|dummyCol|
+---+--------------------+--------------------+---------+----------+-----+----------+--------+
|  1|                Luis|         Alvarez$$%!|     6.15|      Cake|   10|1980/07/07|   never|
|  2|               André|              Ampère|    21.15|      piza|    8|1950/07/08|   gonna|
|  3|               NiELS|          Böhr//((%%|    27.55|     pizza|    8|1990/07/09|    give|
|  4|                PAUL|              dirac$|    26.05|     pizza|    8|1954/07/10|     you|
|  5|              Albert|            Einstein|     31.7|     pizza|    8|1990/07/11|      up|
|  6|             Galileo|             GALiLEI|     33.6|     arepa|    5|1930/08/12|   never|
|  7|                CaRL|            Ga%%%uss|    16.15|      taco|    3|1970/07/13|   gonna|
|  8|               David|          H$$$ilbert|   

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc = spark.sparkContext

spark.createDataFrame(sc.parallelize(range(1, 6)).map(lambda i: Row(single=i, double=i ** 2))).collect()

[Row(double=1, single=1),
 Row(double=4, single=2),
 Row(double=9, single=3),
 Row(double=16, single=4),
 Row(double=25, single=5)]

## Cheat Sheet

In [15]:
df = op.create.df(
    [
        ("words", "str", True),
        ("num", "int", True),
    ],
    [
        ("  I like     fish  ", 1),
        ("    zombies", 2, ),
        ("simpsons   cat lady", 2),
        (None, 3)
    ])

In [34]:
df.sample_by(10, False).table()

id (int),firstName (string),lastName (string),billingId (int),product (string),price (int),birth (string),dummyCol (string)
2,André,Ampère,423,piza,8,1950/07/08,gonna
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down
11,Isaac,Newton,992,pasta,9,1999/02/15,never⸱
14,Fred,Hoy&&&le,553,pizzza,8,1997/06/27,around
16,William,Gilbert###,886,BEER,2,1958/03/26,desert
17,Marie,CURIE,912,Rice,1,2000/03/22,you
18,Arthur,COM%%%pton,812,110790,5,1899/01/01,#


In [60]:
df.cols.select(["firstName",2,3]).show()

+--------------------+--------------------+---------+
|           firstName|            lastName|billingId|
+--------------------+--------------------+---------+
|                Luis|         Alvarez$$%!|      123|
|               André|              Ampère|      423|
|               NiELS|          Böhr//((%%|      551|
|                PAUL|              dirac$|      521|
|              Albert|            Einstein|      634|
|             Galileo|             GALiLEI|      672|
|                CaRL|            Ga%%%uss|      323|
|               David|          H$$$ilbert|      624|
|            Johannes|              KEPLER|      735|
|               JaMES|         M$$ax%%well|      875|
|               Isaac|              Newton|      992|
|              Emmy%%|            Nöether$|      234|
|              Max!!!|           Planck!!!|      111|
|                Fred|            Hoy&&&le|      553|
|(((   Heinrich )))))|               Hertz|      116|
|             William|      

In [55]:
value = ["1"]
bool(value) and isinstance(value, list) and all(isinstance(elem, (int, str)) for elem in value)

True

DataFrame[id: int, firstName: string, lastName: string, billingId: int, product: string, price: int, birth: string, dummyCol: string]

In [85]:
from pyspark.sql import functions as F
c = "firstName"
df.cols.qcut("billingId","b",3).table()

id (int),firstName (string),lastName (string),billingId (int),product (string),price (int),birth (string),dummyCol (string),b (double)
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never,0.0
2,André,Ampère,423,piza,8,1950/07/08,gonna,0.0
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give,1.0
4,PAUL,dirac$,521,pizza,8,1954/07/10,you,1.0
5,Albert,Einstein,634,pizza,8,1990/07/11,up,1.0
6,Galileo,⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱GALiLEI,672,arepa,5,1930/08/12,never,2.0
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna,0.0
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let,1.0
9,Johannes,KEPLER,735,taco,3,1920/04/22,you,2.0
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down,2.0


In [98]:
df.cols.clip("billingId",200, 400).show()

+---+--------------------+--------------------+---------+----------+-----+----------+--------+
| id|           firstName|            lastName|billingId|   product|price|     birth|dummyCol|
+---+--------------------+--------------------+---------+----------+-----+----------+--------+
|  1|                Luis|         Alvarez$$%!|      200|      Cake|   10|1980/07/07|   never|
|  2|               André|              Ampère|      400|      piza|    8|1950/07/08|   gonna|
|  3|               NiELS|          Böhr//((%%|      400|     pizza|    8|1990/07/09|    give|
|  4|                PAUL|              dirac$|      400|     pizza|    8|1954/07/10|     you|
|  5|              Albert|            Einstein|      400|     pizza|    8|1990/07/11|      up|
|  6|             Galileo|             GALiLEI|      400|     arepa|    5|1930/08/12|   never|
|  7|                CaRL|            Ga%%%uss|      323|      taco|    3|1970/07/13|   gonna|
|  8|               David|          H$$$ilbert|   

In [89]:
df.cols.abs("billingId").show()

+---+--------------------+--------------------+---------+----------+-----+----------+--------+
| id|           firstName|            lastName|billingId|   product|price|     birth|dummyCol|
+---+--------------------+--------------------+---------+----------+-----+----------+--------+
|  1|                Luis|         Alvarez$$%!|      123|      Cake|   10|1980/07/07|   never|
|  2|               André|              Ampère|      423|      piza|    8|1950/07/08|   gonna|
|  3|               NiELS|          Böhr//((%%|      551|     pizza|    8|1990/07/09|    give|
|  4|                PAUL|              dirac$|      521|     pizza|    8|1954/07/10|     you|
|  5|              Albert|            Einstein|      634|     pizza|    8|1990/07/11|      up|
|  6|             Galileo|             GALiLEI|      672|     arepa|    5|1930/08/12|   never|
|  7|                CaRL|            Ga%%%uss|      323|      taco|    3|1970/07/13|   gonna|
|  8|               David|          H$$$ilbert|   

In [20]:
from optimus import Optimus

op = Optimus()
df_cancer =op.load.url("https://raw.githubusercontent.com/ironmussa/Optimus/master/tests/data_cancer.csv")

columns = ['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean',
           'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean',
           'fractal_dimension_mean']

df_model, rf_model = op.ml.random_forest(df_cancer, columns, "diagnosis")
df_model.cols.select(["label","prediction"]).table()



label 1 (double),prediction 2 (double)
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
